In [54]:
import urllib
import os
import collections
import numpy as np
from tqdm import tqdm
import torch
from torch import nn
import torchtext
from torch.utils.data import Dataset, DataLoader, random_split

In [55]:
os.makedirs('data', exist_ok=True)
dataset_train, dataset_test = torchtext.datasets.AG_NEWS(root='./data')
tokenizer = torchtext.data.utils.get_tokenizer('basic_english', language="en")
dataset_train = list(dataset_train)
dataset_test = list(dataset_test)

In [56]:
tokenized_data = []
label_data = []
counter = collections.Counter()

print('Start Tokenizing...')
for label, line in tqdm(dataset_train):
    tokens = tokenizer(line)
    tokenized_data.append(tokens)
    label_data.append(label-1)

print('Making Vocab...')
for line in tqdm(tokenized_data):
    counter.update(line)

vocab = torchtext.vocab.vocab(counter, min_freq=1, specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])

Start Tokenizing...


100%|██████████| 120000/120000 [00:04<00:00, 26646.58it/s]


Making Vocab...


100%|██████████| 120000/120000 [00:01<00:00, 91342.75it/s]


In [57]:
class AGNewsDataset(Dataset):
    def __init__(self, tokenized_data, label_data, max_seq = 256):
        self.x = []
        self.y = []
        self.classes = ['World', 'Sports', 'Business', 'Sci/Tech']
        
        for tokens in tqdm(tokenized_data):
            token_ids = [vocab[token] for token in tokens]
            self.x.append(token_ids)
        self.y = label_data
        
    def __len__(self):
        return len(self.x)
        
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]) , torch.tensor(self.y[idx])

In [58]:
news_dataset = AGNewsDataset(tokenized_data, label_data, max_seq=256)

100%|██████████| 120000/120000 [00:04<00:00, 24156.25it/s]


In [59]:
train_ratio = 0.8
train_size = int(train_ratio*len(news_dataset))
valid_size = len(news_dataset) - train_size
train_dataset, valid_dataset = random_split(news_dataset, [train_size, valid_size])

In [60]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.rnn = nn.RNN(embedding_size, hidden_size, batch_first=True)
        self.out =  nn.Linear(hidden_size, output_size)

    def forward(self, input):
        input = self.embedding(input)
        #input: [batch_size, sentence_len, emb_size]
        output, _ = self.rnn(input)
        #output: [batch_size, sentence_len, hidden_size]
        output = self.out(output)
        #output: [batch_size, sentence_len, output_size]
        output = output.transpose(1,0)
        #output: [sentence_len, batch_size, output_size]
        output = output[-1]
        #output: [batch_size, output_size]

        return output



In [61]:
model = RNN(vocab_size=len(vocab), embedding_size=64, hidden_size=32, output_size=len(news_dataset.classes))

In [62]:
batch_size=256
learning_rate = 0.001
num_epochs = 10
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [63]:
def pad_collate_fn(batch):
    collate_x = []
    collate_y = []
    for data, label in batch:
        collate_x.append(data)
        collate_y.append(label)
    collate_x = nn.utils.rnn.pad_sequence(collate_x, padding_value=vocab['<pad>'], batch_first=True)
    collate_y = torch.stack(collate_y)
    return (collate_x, collate_y)



In [64]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= pad_collate_fn)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn= pad_collate_fn)

In [65]:
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [66]:
def accuracy(output, label):
    pred = torch.argmax(output, dim=-1)
    acc = torch.sum(pred==label)
    return acc
    

In [67]:
def train(dataloader, epoch):
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    data_num = 0
    for data,label in tqdm(dataloader, desc=f"Epoch {epoch}"):
        data, label = data.to(device), label.to(device)
        output = model(data)
        optimizer.zero_grad()
        loss = criterion(output, label)
        acc = accuracy(output, label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        data_num += data.size(0)

    print(f"Train Epoch: {epoch}, Loss: {epoch_loss/len(dataloader)}, Acc: {epoch_acc/data_num}")


In [68]:
def evaluate(dataloader, epoch):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    data_num = 0
    with torch.no_grad():
        for data,label in tqdm(dataloader, desc=f"Epoch {epoch}"):
            data, label = data.to(device), label.to(device)
            output = model(data)
            loss = criterion(output, label)
            acc = accuracy(output, label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            data_num += data.size(0)

    print(f"Evaluate Epoch: {epoch}, Loss: {epoch_loss/len(dataloader)}, Acc: {epoch_acc/data_num}")
    return epoch_acc/data_num

In [69]:
total_acc = 0
best_acc = 0
for epoch in range(num_epochs):
    train(train_dataloader, epoch)
    acc = evaluate(valid_dataloader, epoch)
    best_acc = max(acc, best_acc)
    print('-'*50)

print(f"Best Accuracy: {best_acc}")



Epoch 0: 100%|██████████| 375/375 [00:06<00:00, 57.04it/s]


Train Epoch: 0, Loss: 1.3877261079152425, Acc: 0.24947916666666667


Epoch 0: 100%|██████████| 94/94 [00:00<00:00, 115.19it/s]


Evaluate Epoch: 0, Loss: 1.3866007315351607, Acc: 0.25225
--------------------------------------------------


Epoch 1: 100%|██████████| 375/375 [00:06<00:00, 56.64it/s]


...

Evaluate Epoch: 8, Loss: 1.3852464467921155, Acc: 0.25154166666666666
--------------------------------------------------


Epoch 9: 100%|██████████| 375/375 [00:06<00:00, 56.67it/s]


Train Epoch: 9, Loss: 1.384731083869934, Acc: 0.253375


Epoch 9: 100%|██████████| 94/94 [00:00<00:00, 99.63it/s] 

Evaluate Epoch: 9, Loss: 1.3852993643030207, Acc: 0.24966666666666668
--------------------------------------------------
Best Accuracy: 0.25233333333333335
